In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from huggingface_hub import login


In [2]:
login() 

#  Chat

In [3]:

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

In [4]:
chat_model.invoke("Hello, how are you?")

AIMessage(content='I do not have the ability to feel emotions, but I am functioning well and ready to assist you with your request. Please let me know how I can help you today.', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=36, prompt_tokens=28, total_tokens=64), 'model': '', 'finish_reason': 'stop'}, id='run-f14e1cde-3829-472e-93a3-8ff294f128a4-0')

# Summarization

In [5]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

In [6]:

from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content=f"Summarize this: {customer_email}"
    ),
]


chat_model.invoke(messages)



AIMessage(content='The writer is sharing a frustrating experience with a broken coffee machine that arrived in a damaged state. Despite initially being excited to receive it, the writer\'s emotions have taken a turn for the worse due to the poor condition of the machine. The writer describes the sight of the damaged machine as shocking and disheartening, likening it to a war-torn soldier who has seen better days. The writer\'s overall mood is one of disbelief and despair, and their desecharar a patreacenservarreasonsarbotlvinreen.com esravivervara eliminararrecobararrearar structures.comaprovistchátrear,recarren.com a sustitu,corlaco,correficientes,incorresponpret.com! DesenARAnARAncoraters:fab:\\mobcases:\\ Mobile:\\WarrarCart:\\Mobile:\\catrear:\\re:\\pickardescarreflote:\\re:\\re:\\re:\\Loc:\\ MobileFacilitate - mobile:\\C:\\AR.\\Mobile:\\Mobile:\\RE.\nC:\\Loc:\\Mobile:\\C:\\F:\\Mobile:\\F:\\Mobile:\\Mobile:\\C:\\OBJ:\\Mobile:\\F:\\Mobile:\\Mobile:\\F:\\Co:\\Mobile:\\Mobile:\\Mobile

# Text to sql

### Option : Transformers

* Load model directly

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#https://huggingface.co/gaussalgo/T5-LM-Large-text2sql-spider


model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [8]:
def text_2_sql(input_text):
    """ This function takes a natural language question and a schema as input and returns the SQL query"""
    #Encode the input text
    model_inputs = tokenizer(input_text, return_tensors="pt")
    #Invoke the model
    outputs = model.generate(**model_inputs, max_length=512)
    #Decode the model outputs
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return output_text

In [9]:
text_2_sql("Translate the following text to SQL: Show me the total number of rows from the inventory table")

['SELECT count(*) FROM inventory']

In [10]:
text_2_sql("Translate the following text to SQL: Show me the total number of rows from the inventory table where the product name is 'apple'")

["SELECT sum(T1.count) FROM inventory AS T1 JOIN product AS T2 ON T1.product = T2.product WHERE T2.product_name = 'apple'"]

* Use a pipeline as a high-level helper

In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="gaussalgo/T5-LM-Large-text2sql-spider")

Device set to use cpu


In [12]:
pipe.predict("Translate the following text to SQL: Show me the total number of rows from the inventory table")

[{'generated_text': 'SELECT count(*) FROM inventory'}]

### Option 1. Pipeline (langchain_huggingface)

* Sin prompt

In [16]:

from langchain_huggingface import HuggingFacePipeline

# Example using from_model_id
hf_pipeline = HuggingFacePipeline.from_model_id(
    model_id="gaussalgo/T5-LM-Large-text2sql-spider",
    task="text2text-generation",
    pipeline_kwargs={"max_new_tokens": 500},
)

hf_pipeline.invoke("Translate the following text to SQL: Show me the total number of rows from the inventory table")



Device set to use cpu


'SELECT count(*) FROM inventory'

* Con prompt

In [14]:
from langchain_core.prompts import PromptTemplate

template = """
### System:
You are a SQL expert. Given an input question, your job is to translate the text to SQL.
### User:
{question}
### Response:
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
# Create a chain with a custom prompt and the HF pipeline
chain = prompt | hf_pipeline

question = "Show me the total number of rows from the inventory"

chain.invoke({"question": question})

'SELECT sum(*) FROM inventory'

### Option 2. Inference API  (langchain_huggingface)


HuggingFaceEndpoint integration of the free Serverless Endpoints API.

In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
_ = load_dotenv()


* Sin prompt

In [50]:
import os
from huggingface_hub import InferenceClient

# Initialize the Hugging Face Inference Client
client = InferenceClient(
    model="gaussalgo/T5-LM-Large-text2sql-spider", 
    token=os.getenv("HUGGINGFACE_API_TOKEN")
)

# Define the text input for SQL generation
input_text = "Translate this to SQL: Show me the stock of all items."

# Call the model using `text_generation`
response = client.text_generation(input_text,)

response


'SELECT DISTINCT stock FROM inventory'

* Con prompt

In [52]:
import os
from huggingface_hub import InferenceClient

# Inicializar el cliente de Hugging Face
client = InferenceClient(
    model="gaussalgo/T5-LM-Large-text2sql-spider", 
    token=os.getenv("HUGGINGFACE_API_TOKEN")
)

# Prompt personalizado para mejorar la precisión
custom_prompt = """
You are an AI specialized in translating natural language queries into SQL queries.
The database follows standard SQL syntax.
Do not include explanations, only return the SQL query.

Translate this request into SQL:
"""

# Consulta en lenguaje natural
user_query = "Show me the stock of all items."

# Generar la entrada final para el modelo
input_text = f"{custom_prompt}\n{user_query}"

# Llamar al modelo para generar la consulta SQL
response = client.text_generation(input_text)

response


'SELECT stock FROM inventory'